In [1]:
import re
import time
import json
import uuid
import redis
import datetime
import pandas as pd

from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
pool = redis.ConnectionPool(host='172.17.0.2', port=6379, decode_responses=True, db=1)
redis_con = redis.Redis(connection_pool=pool)

addr = '172.17.0.3'
user = 'root'
passwd = '123456'

mysql_engin = 'mysql+pymysql://'+user+':'+passwd+'@'+addr+'/hot?charset=utf8'
engine = create_engine(mysql_engin)

In [9]:
def save_links_to_redis(theme):
    # 清空数据库
    redis_con.flushdb()
    for i in range(0, len(theme)):
        url = theme['url'][i]
        redis_con.sadd('theme', url)
        
theme = pd.read_csv('theme.csv')
theme = theme.loc[(theme['platform'] == 'tianmao'),:].reset_index(drop=True)
save_links_to_redis(theme)

# 一.获取品类链接

In [ ]:
import configparser
cf = configparser.ConfigParser()
cf.read("config/crawl.cfg")

def match_chinese(string):
    return re.findall('[\u4e00-\u9fa5]+',string)[0]

data_theme = []
data_son_theme = []
data_son_theme_url = []
data_platform = []

profile = webdriver.FirefoxProfile('/home/ml/.mozilla/firefox/0k5sh6ex.default')
driver = webdriver.Firefox(profile)
for i,platform in enumerate(cf.options('url')):
    driver.get(cf.get('url', platform))
    parse_theme = platform + '_parse_theme'
    
    while len(driver.find_elements_by_xpath(cf.get(parse_theme,'move_to_element'))) == 0:
        pass
    ele = driver.find_elements_by_xpath(cf.get(parse_theme,'move_to_element'))[0]
    ActionChains(driver).move_to_element(ele).perform()

    while len(driver.find_elements_by_xpath(cf.get(parse_theme,'themes'))) == 0:
        pass
        
    themes = driver.find_elements_by_xpath(cf.get(parse_theme,'themes'))
    for i, theme in enumerate(themes):
        theme_text = match_chinese(theme.get_attribute('textContent'))
        son_parse = cf.get(parse_theme,'son_themes').split('+str(i+1)+')
        son_themes_text = driver.find_elements_by_xpath(son_parse[0]+str(i+1)+son_parse[1])
        for j, son_theme in enumerate(son_themes_text):
            son_theme_text = match_chinese(son_theme.get_attribute('textContent'))
            url = son_theme.get_attribute(cf.get(parse_theme,'attri_url'))
            data_theme.append(theme_text)
            data_son_theme.append(son_theme_text)
            data_son_theme_url.append(url+'&'+cf.get(parse_theme,'sales_volume'))
            data_platform.append(platform)
woman_cloth_links = pd.DataFrame(pd.Series(data_theme), columns=['theme'])
woman_cloth_links['son_theme'] = pd.Series(data_son_theme)
woman_cloth_links['url'] = pd.Series(data_son_theme_url)
woman_cloth_links['platform'] = data_platform
woman_cloth_links.to_csv('theme.csv', index=False)

# 二.获取商品链接

In [6]:
import configparser
cf = configparser.ConfigParser()
cf.read("config/crawl.cfg")

['config/crawl.cfg']

In [10]:
profile = webdriver.FirefoxProfile('/home/ml/.mozilla/firefox/0k5sh6ex.default')
driver = webdriver.Firefox(profile)

In [12]:
time1 = time.time()
current_url = ''

while True:
    url = redis_con.spop('theme')
    platform_parse = re.findall('\.(.*?).com',url)[0]+'_parse_links'
    items_list = cf.options(platform_parse)
    items_list.pop(0)
    items_list.pop(0)
    
    current_url = url
    driver.get(url)
    page_nums = int(driver.find_elements_by_xpath(cf.get(platform_parse,'page_num')[:-1])[0].text.split(cf.get(platform_parse,'page_num')[-1])[1])
    page_nums = 5
    final = pd.DataFrame([],columns=items_list)
    data = pd.DataFrame([],columns=items_list)
    for page_num in range(0,page_nums):
        # 等待信息装载
        while True:                   
            result = [len(driver.find_elements_by_xpath(cf.get(platform_parse, item))) for item in items_list]
            if (0 not in result)&(len(set(result)) == 1):
                driver.execute_script("document.documentElement.scrollTop="+str(100))
                time.sleep(5)
                break

        # 提取信息
        while True:
            try:
                for item in items_list:
                    if 'attribute' in item:
                        temp = cf.get(platform_parse, item).split('|')
                        a = driver.find_elements_by_xpath(temp[0])
                        data[item] = pd.Series([a[i].get_attribute(temp[1]) for i in range(0, len(a))])
                    else:
                        a = driver.find_elements_by_xpath(cf.get(platform_parse, item))
                        data[item] = pd.Series([a[i].text for i in range(0, len(a))])
                data['id'] = pd.Series([str(uuid.uuid4()) for i in range(0, len(a))])
                data['page_num'] = page_num
            except:
                continue
            break
        
        # 最后一页退出
        if page_num == (page_nums-1):
            break
            
        # 翻页,并等待页面刷新
        driver.find_elements_by_xpath(cf.get(platform_parse, 'click'))[0].click()
        while current_url == driver.current_url:
            pass
        
        # 超过一半重复退出
        current_num = len(final)
        final = final.append(data).drop_duplicates(items_list).reset_index(drop=True)
        if len(final) < current_num + int(len(data)/2):
            break
    final.to_sql('links', con=engine,  if_exists='append', index=False)

print(time.time()-time1)
final.to_csv('links.csv', index=False)

WebDriverException: Message: Failed to interpret value as array


In [8]:
redis_con.sadd('theme', url)

1

# 三.下载图片

In [ ]:
import requests
from io import BytesIO
from PIL import Image

items = pd.read_csv('links.csv')
for i in range(0,len(items)):
    try:
        src = 'https:'+items['images_url_attribute'][i]
        dir_path = 'data/samples/'+items['id'][i]+'.jpg'
        print(src)
        pic = Image.open(BytesIO(requests.get(src).content))
        pic = pic.convert('RGB')
        w, h = pic.size
        pic = pic.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
        pic.save(dir_path)
    except requests.exceptions.ConnectionError:
        print('Sorrry,image cannot downloaded, url is error{}.'.format(src))